<a href="https://colab.research.google.com/github/burnout13l/Tesis_R-V/blob/main/datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalar librerías necesarias
!pip install PyPDF2

import re
import PyPDF2
import os
import pandas as pd

# Carpeta donde están los PDF
pdf_folder = "/content/datos/"

# Lista donde guardaremos todos los diccionarios
data_list = []

# Función para extraer datos de un PDF
def extract_data(pdf_path):
    with open(pdf_path, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        text = ""
        for page in reader.pages:
            text += page.extract_text() + "\n"

    # Diccionario con los campos
    data = {
        "archivo": os.path.basename(pdf_path),
        "cliente": None,
        "instrumento": None,
        "marca": None,
        "modelo": None,
        "rango": None,
        "temp": None,
        "humedad": None,
        "lugar": None,
        "error": None,
        "incertidumbre": None,
        "fecha_calibracion": None,
        "periodicidad": None
    }

    # Cliente
    cliente = re.search(r"([A-ZÑ\s]+S\.A\.)", text)
    if cliente:
        data["cliente"] = cliente.group(1)

    # Instrumento
    instr = re.search(r"Denominación:\s*(.*)", text)
    if instr:
        data["instrumento"] = instr.group(1).strip()

    # Marca
    marca = re.search(r"Marca:\s*([A-Za-z0-9]+)", text)
    if marca:
        data["marca"] = marca.group(1)

    # Modelo
    modelo = re.search(r"Modelo:\s*([A-Za-z0-9*]+)", text)
    if modelo:
        data["modelo"] = modelo.group(1)

    # Rango
    rango = re.search(r"Rango:\s*\((.*?)\)", text)
    if rango:
        data["rango"] = rango.group(1)

    # Temperatura
    temp = re.search(r"(\d+\s*±\s*\d*)\s*ºC", text)
    if temp:
        data["temp"] = temp.group(0)

    # Humedad
    humedad = re.search(r"(\d+\s*±\s*\d*)\s*%RH", text)
    if humedad:
        data["humedad"] = humedad.group(0)

    # Lugar
    lugar = re.search(r"Lugar de calibración:\s*(.*)", text)
    if lugar:
        data["lugar"] = lugar.group(1).strip()

    # Error
    if "Errores de Medición" in text:
        data["error"] = "Incluido en certificado"

    # Incertidumbre
    if "Incertidumbre" in text:
        data["incertidumbre"] = "Detallada en certificado"

    # Fecha calibración
    fecha = re.search(r"Fecha de emisión del Certificado:\s*(\d{4}\s*-\s*\d{2}\s*-\s*\d{2})", text)
    if fecha:
        data["fecha_calibracion"] = fecha.group(1)

    # Periodicidad
    periodicidad = re.search(r"Periodicidad.*?:\s*(.*)", text)
    if periodicidad:
        data["periodicidad"] = periodicidad.group(1).strip()

    return data


# Recorrer todos los PDF en la carpeta y guardarlos en la lista
for file in os.listdir(pdf_folder):
    if file.endswith(".pdf"):
        pdf_path = os.path.join(pdf_folder, file)
        diccionario = extract_data(pdf_path)
        data_list.append(diccionario)

# Convertir lista de diccionarios en DataFrame
df = pd.DataFrame(data_list)

# Mostrar resultados organizados
df


,archivo,cliente,instrumento,marca,modelo,rango,temp,humedad,lugar,error,incertidumbre,fecha_calibracion,periodicidad
0,P.41.04.11 - E2.DV2-signed-signed.pdf,EL ORDEÑO S.A.,Manómetro,TETRA,*******,0 a 10,025 ± 1 ºC,61 ± 1 %RH,En Sitio,Incluido en certificado,Detallada en certificado,2025 - 02 - 06,* 12 Meses
1,P.41.04.10 - E2.DV1-signed-signed.pdf,EL ORDEÑO S.A.,Manómetro,REOTEMP,*******,0 a 200,926 ± 1 ºC,53 ± 1 %RH,En Sitio,Incluido en certificado,Detallada en certificado,2025 - 02 - 06,* 12 Meses
2,P.41.04.08 - E2.TV-signed-signed.pdf,EL ORDEÑO S.A.,Vacuómetro,WIKA,*******,-30 a 0,3925 ± 1 ºC,52 ± 1 %RH,En Sitio,Incluido en certificado,Detallada en certificado,2025 - 02 - 06,* 12 Meses
3,P.41.04.12 - E2.DV-signed-signed.pdf,EL ORDEÑO S.A.,Manómetro,USG,*******,0 a 300,625 ± 1 ºC,61 ± 1 %RH,En Sitio,Incluido en certificado,Detallada en certificado,2025 - 02 - 06,* 12 Meses
4,P.41.04.16 - TS.IN.PI03-signed-signed.pdf,EL ORDEÑO S.A.,Manómetro,WINTERS,*******,0 a 80,626 ± 1 ºC,61 ± 1 %RH,En Sitio,Incluido en certificado,Detallada en certificado,2025 - 02 - 06,* 12 Meses
5,P.41.04.07 - EW.IN.PI07-signed-signed.pdf,EL ORDEÑO S.A.,Manómetro,ITEC,*******,0 a 200,426 ± 1 ºC,59 ± 1 %RH,En Sitio,Incluido en certificado,Detallada en certificado,2025 - 02 - 06,* 12 Meses


In [ ]:
# Instalar librerías necesarias
!pip install PyPDF2

import re
import PyPDF2
import os
import pandas as pd

# Carpeta donde están los PDF (ej: /content/datos/)
pdf_folder = "/content/datos/"

# Lista para guardar los resultados
rows = []

# Función para extraer datos de un solo PDF
def extract_data(pdf_path):
    with open(pdf_path, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        text = ""
        for page in reader.pages:
            text += page.extract_text() + "\n"

    data = {
        "archivo": os.path.basename(pdf_path),
        "cliente": None,
        "instrumento": None,
        "marca": None,
        "modelo": None,
        "rango": None,
        "temp": None,
        "humedad": None,
        "lugar": None,
        "error": None,
        "incertidumbre": None,
        "fecha_calibracion": None,
        "periodicidad": None
    }

    # Cliente
    cliente = re.search(r"([A-ZÑ\s]+S\.A\.)", text)
    if cliente:
        data["cliente"] = cliente.group(1)

    # Instrumento
    instr = re.search(r"Denominación:\s*(.*)", text)
    if instr:
        data["instrumento"] = instr.group(1).strip()

    # Marca
    marca = re.search(r"Marca:\s*([A-Za-z0-9]+)", text)
    if marca:
        data["marca"] = marca.group(1)

    # Modelo
    modelo = re.search(r"Modelo:\s*([A-Za-z0-9*]+)", text)
    if modelo:
        data["modelo"] = modelo.group(1)

    # Rango
    rango = re.search(r"Rango:\s*\((.*?)\)", text)
    if rango:
        data["rango"] = rango.group(1)

    # Temperatura
    temp = re.search(r"(\d+\s*±\s*\d*)\s*ºC", text)
    if temp:
        data["temp"] = temp.group(0)

    # Humedad
    humedad = re.search(r"(\d+\s*±\s*\d*)\s*%RH", text)
    if humedad:
        data["humedad"] = humedad.group(0)

    # Lugar
    lugar = re.search(r"Lugar de calibración:\s*(.*)", text)
    if lugar:
        data["lugar"] = lugar.group(1).strip()

    # Error (no es un número exacto, solo indicador)
    if "Errores de Medición" in text:
        data["error"] = "Incluido en certificado"

    # Incertidumbre
    if "Incertidumbre" in text:
        data["incertidumbre"] = "Detallada en certificado"

    # Fecha calibración
    fecha = re.search(r"Fecha de emisión del Certificado:\s*(\d{4}\s*-\s*\d{2}\s*-\s*\d{2})", text)
    if fecha:
        data["fecha_calibracion"] = fecha.group(1)

    # Periodicidad
    periodicidad = re.search(r"Periodicidad.*?:\s*(.*)", text)
    if periodicidad:
        data["periodicidad"] = periodicidad.group(1).strip()

    return data

# Recorrer todos los PDF de la carpeta
for file in os.listdir(pdf_folder):
    if file.endswith(".pdf"):
        pdf_path = os.path.join(pdf_folder, file)
        row = extract_data(pdf_path)
        rows.append(row)

# Convertir resultados a DataFrame
df = pd.DataFrame(rows)

# Mostrar tabla
df

# Guardar a Excel o CSV
df.to_excel("/content/resultados_calibracion.xlsx", index=False)
df.to_csv("/content/resultados_calibracion.csv", index=False)
